In [1]:
import numpy as np
import pandas as pd

In [2]:
%matplotlib inline

In [3]:
file_path = '/content/drive/MyDrive/Solar energy prediction - Project/Datasets/Out/IDF_solar_production_hourly_2017-2020.csv'

In [5]:
df = pd.read_csv(file_path, parse_dates=['Date - Heure'], index_col='Date - Heure')

In [6]:
df.head()

,Solaire (MW)
Date - Heure,
2017-01-01 00:00:00,0
2017-01-01 01:00:00,0
2017-01-01 02:00:00,0
2017-01-01 03:00:00,0
2017-01-01 04:00:00,0


**Modify the dataset with the same timestamp as three-hourly solar radiance dataset to ease the merge of all dataset**

In [7]:
df1 = df.loc['2017-01-01 01:00:00': '2017-03-26 01:00:00']

In [8]:
df2 = df.loc['2017-03-26 05:00:00': '2017-10-29 02:00:00']
df3 = df.loc['2017-10-29 04:00:00': '2018-03-25 01:00:00']
df4 = df.loc['2018-03-25 05:00:00': '2018-10-28 02:00:00']
df5 = df.loc['2018-10-28 04:00:00': '2019-03-31 01:00:00']
df6 = df.loc['2019-03-31 05:00:00': '2019-10-27 02:00:00']
df7 = df.loc['2019-10-27 04:00:00': '2020-03-29 01:00:00']
df8 = df.loc['2020-03-29 05:00:00': '2020-10-25 02:00:00']
df9 = df.loc['2020-10-25 04:00:00': ]

In [9]:
df1 = df1.resample('3H', origin='start').sum()
df2 = df2.resample('3H', origin='start').sum()
df3 = df3.resample('3H', origin='start').sum()
df4 = df4.resample('3H', origin='start').sum()
df5 = df5.resample('3H', origin='start').sum()
df6 = df6.resample('3H', origin='start').sum()
df7 = df7.resample('3H', origin='start').sum()
df8 = df8.resample('3H', origin='start').sum()
df9 = df9.resample('3H', origin='start').sum()

In [10]:
df_production = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8, df9])

In [11]:
df_production.head()

,Solaire (MW)
Date - Heure,
2017-01-01 01:00:00,0
2017-01-01 04:00:00,0
2017-01-01 07:00:00,0
2017-01-01 10:00:00,15
2017-01-01 13:00:00,22


In [12]:
df_production.to_csv('/content/drive/MyDrive/Solar energy prediction - Project/Datasets/Out/IDF_soloar_production_3hourly.csv')

In [21]:
df_weather = pd.read_csv('/content/drive/MyDrive/Solar energy prediction - Project/Datasets/Out/weather_preprocessed.csv', 
                         parse_dates=['DATE'],
                         index_col='DATE')
df_weather.head()

,wind_direction_angle,wind_speed_rate,air_temp,precip_depth_per_hour,station_pressure_rate
DATE,,,,,
2017-01-01 00:00:00,180,31,-39,0.0,10188
2017-01-01 01:00:00,210,21,-37,0.0,10180
2017-01-01 02:00:00,180,15,-37,0.0,10174
2017-01-01 03:00:00,190,15,-39,0.0,10169
2017-01-01 04:00:00,200,26,-40,0.0,10162


In [22]:
df_radiance = pd.read_csv('/content/drive/MyDrive/Solar energy prediction - Project/Datasets/Out/idf_2017-2020_three-hourly_solar_radiance.csv', 
                          parse_dates=['Date'], 
                          index_col='Date')
df_radiance.head()

,Vitesse du vent à 100m (m/s),Rayonnement solaire global (W/m2)
Date,,
2017-01-01 01:00:00,4.53,0.00
2017-01-01 04:00:00,5.30,0.00
2017-01-01 07:00:00,4.92,0.00
2017-01-01 10:00:00,4.71,7.09
2017-01-01 13:00:00,4.30,84.24


In [23]:
#remove duplicated values in the radiance dataset
df_radiance = df_radiance[~df_radiance.index.duplicated()]

In [30]:
#add the missing data for 2018-01-01 01:00:00 taking the average of the observations above and below 
df_radiance.loc['2018-01-01 01:00:00'] = [1/2 *(df_radiance.loc['2018-01-01 04:00:00']['Vitesse du vent à 100m (m/s)']
                                                + df_radiance.loc['2017-12-31 22:00:00']['Vitesse du vent à 100m (m/s)']),
                                          1/2 *(df_radiance.loc['2018-01-01 04:00:00']['Rayonnement solaire global (W/m2)']
                                                + df_radiance.loc['2017-12-31 22:00:00']['Rayonnement solaire global (W/m2)'])]

In [40]:
df_radiance.count()

Vitesse du vent à 100m (m/s)         11688
Rayonnement solaire global (W/m2)    11688
dtype: int64

In [48]:
df_radiance.index = pd.to_datetime(df_radiance.index)

In [24]:
#wind speed already existing in solar radiance dataset, direction angle shouldn't be of great importance
df_weather.drop(columns=['wind_direction_angle', 'wind_speed_rate'], inplace=True)

In [25]:
df_pro_weather = pd.merge(df_production,df_weather,how='left', left_index=True, right_index=True )

In [26]:
df_production.count()

Solaire (MW)    11688
dtype: int64

In [27]:
df_pro_weather.count()

Solaire (MW)             11688
air_temp                 11455
precip_depth_per_hour    11455
station_pressure_rate    11455
dtype: int64

In [28]:
#There are missing values of weather when consolidating the production and weather data
df_pro_weather[df_pro_weather.isna().any(axis=1)]

,Solaire (MW),air_temp,precip_depth_per_hour,station_pressure_rate
Date - Heure,,,,
2017-01-18 19:00:00,0,NaN,NaN,NaN
2017-04-14 14:00:00,244,NaN,NaN,NaN
2017-05-16 14:00:00,274,NaN,NaN,NaN
2017-05-16 17:00:00,148,NaN,NaN,NaN
2017-05-21 14:00:00,276,NaN,NaN,NaN
...,...,...,...,...
2020-11-11 22:00:00,0,NaN,NaN,NaN
2020-11-12 01:00:00,0,NaN,NaN,NaN
2020-11-12 04:00:00,0,NaN,NaN,NaN


In [33]:
#add the missing hours for the weather datasets
weather_index = pd.date_range(start='2017-01-01 00:00:00', end='2020-12-31 23:00:00', freq='H')
df_weather_rein = df_weather.reindex(weather_index)

In [34]:
#fill the missing data in the weather dataset with the observations above
df_weather_rein.fillna(method='ffill', inplace=True)

In [35]:
df_weather_rein.loc['2017-01-18 19:00:00']

air_temp                    -1.0
precip_depth_per_hour        0.0
station_pressure_rate    10245.0
Name: 2017-01-18 19:00:00, dtype: float64

In [36]:
df_pro_weather2 = pd.merge(df_production,df_weather_rein,how='inner', left_index=True, right_index=True )

In [37]:
df_pro_weather2.count()

Solaire (MW)             11688
air_temp                 11688
precip_depth_per_hour    11688
station_pressure_rate    11688
dtype: int64

In [50]:
df_full = pd.merge(df_pro_weather2, df_radiance,how='inner', left_index=True, right_index=True )

In [51]:
df_full.loc['2018-01-01 01:00:00']

Solaire (MW)                            0.00
air_temp                               89.00
precip_depth_per_hour                   0.00
station_pressure_rate                9984.00
Vitesse du vent à 100m (m/s)           11.41
Rayonnement solaire global (W/m2)       0.00
Name: 2018-01-01 01:00:00, dtype: float64

In [52]:
df_full.count()

Solaire (MW)                         11688
air_temp                             11688
precip_depth_per_hour                11688
station_pressure_rate                11688
Vitesse du vent à 100m (m/s)         11688
Rayonnement solaire global (W/m2)    11688
dtype: int64

In [53]:
df_full.to_csv('/content/drive/MyDrive/Solar energy prediction - Project/Datasets/Out/full.csv',  header=True)